In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

from braindecode.torch_ext.util import np_to_var


In [ ]:
train_sorted_file_names, train_labels = get_all_sorted_file_names_and_labels('train')
print(len(train_sorted_file_names))

train_infos = [get_info_with_mne(fname) for fname in train_sorted_file_names]

In [ ]:
eval_sorted_file_names, eval_labels = get_all_sorted_file_names_and_labels('eval')

print(len(eval_sorted_file_names))

eval_infos = [get_info_with_mne(fname) for fname in eval_sorted_file_names]

In [ ]:
#edf_file, sampling_frequency, n_samples, n_signals, signal_names, duration

In [ ]:
plt.plot(np.array([info[5] for info in eval_infos]) / 60.0)

In [ ]:
plt.plot(np.array([info[1] for info in eval_infos]))

In [ ]:
from autodiag.dataset import DiagnosisSet, get_all_sorted_file_names_and_labels

from autoeeglukas.utils.my_io import get_info_with_mne

In [ ]:

from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.datautil.iterators import get_balanced_batches

def create_set(X, y, inds):
    """
    X list and y nparray
    :return: 
    """
    new_X = []
    for i in inds:
        new_X.append(X[i])
    new_y = y[inds]
    return SignalAndTarget(new_X, new_y)

class TrainValidTestSplitter(object):
    def __init__(self, n_folds, i_test_fold):
        self.n_folds = n_folds
        self.i_test_fold = i_test_fold

    def split(self, X, y):
        if len(X) < self.n_folds:
            raise ValueError("Less Trials: {:d} than folds: {:d}".format(
                len(X), self.n_folds
            ))
        folds = get_balanced_batches(len(X), None, False,
                                     n_batches=self.n_folds)
        test_inds = folds[self.i_test_fold]
        valid_inds = folds[self.i_test_fold - 1]
        all_inds = list(range(len(X)))
        train_inds = np.setdiff1d(all_inds, np.union1d(test_inds, valid_inds))
        assert np.intersect1d(train_inds, valid_inds).size == 0
        assert np.intersect1d(train_inds, test_inds).size == 0
        assert np.intersect1d(valid_inds, test_inds).size == 0
        assert np.array_equal(np.sort(
            np.union1d(train_inds, np.union1d(valid_inds, test_inds))),
            all_inds)

        train_set = create_set(X, y, train_inds)
        valid_set = create_set(X, y, valid_inds)
        test_set = create_set(X, y, test_inds)

        return train_set, valid_set, test_set


class TrainValidSplitter(object):
    def __init__(self, n_folds, i_valid_fold):
        self.n_folds = n_folds
        self.i_valid_fold = i_valid_fold

    def split(self, X, y):
        if len(X) < self.n_folds:
            raise ValueError("Less Trials: {:d} than folds: {:d}".format(
                len(X), self.n_folds
            ))
        folds = get_balanced_batches(len(X), None, False,
                                     n_batches=self.n_folds)
        valid_inds = folds[self.i_valid_fold]
        all_inds = list(range(len(X)))
        train_inds = np.setdiff1d(all_inds, valid_inds)
        assert np.intersect1d(train_inds, valid_inds).size == 0
        assert np.array_equal(np.sort(np.union1d(train_inds, valid_inds)),
            all_inds)

        train_set = create_set(X, y, train_inds)
        valid_set = create_set(X, y, valid_inds)
        return train_set, valid_set

In [ ]:
test_on_eval = True
only_return_exp = False
n_recordings = 10
max_recording_mins = 35
preproc_functions = []
n_folds = 10
i_test_fold = 9

In [ ]:
dataset = DiagnosisSet(n_recordings=n_recordings,
                    max_recording_mins=max_recording_mins,
                    preproc_functions=preproc_functions,
                      train_or_eval='train')
if test_on_eval:
    test_set = DiagnosisSet(n_recordings=n_recordings,
                           max_recording_mins=None,
                           preproc_functions=preproc_functions,
                               train_or_eval='eval')
if not only_return_exp:
    X,y = dataset.load()
    if test_on_eval:
        test_X, test_y = test_set.load()

In [ ]:
if not test_on_eval:
    splitter = TrainValidTestSplitter(n_folds, i_test_fold,)
else:
    splitter = TrainValidSplitter(n_folds, i_valid_fold=i_test_fold)
if not only_return_exp:
    if not test_on_eval:
        train_set, valid_set, test_set = splitter.split(X,y)
    else:
        train_set, valid_set = splitter.split(X,y)
        test_set = SignalAndTarget(test_X, test_y)
    
del X,y
del test_X, test_y
    
    
# if not test on eval
#exp.test_set = test_set